In [144]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from glob import glob

In [145]:
%cd C:\Users\luzhbin.aleksey\Desktop\analytics

C:\Users\luzhbin.aleksey\Desktop\analytics


**Загрузка данных: должники в апреле**

In [146]:
%cd Overdue_apr
path = glob('*.csv')

deb_apr = pd.read_csv(path[0], sep = ';',
                dtype = {'LoanSum': 'float'},
                names = ['Dog', 'FIO', 'LoanDate', 'Sum', 'Status', 'DaysOverdue', 'OverdueDate'],
                usecols = [0, 2, 12, 13, 14, 15, 16], encoding = 'cp1251')

for i in range(1, len(path)):
    tmp = pd.read_csv(path[i], sep = ';',
                dtype = {'LoanSum': 'float'},
                names = ['Dog', 'FIO', 'LoanDate', 'Sum', 'Status', 'DaysOverdue', 'OverdueDate'],
                usecols = [0, 2, 12, 13, 14, 15, 16], encoding = 'cp1251')
    deb_apr = deb_apr.append(tmp, ignore_index = True)

print(deb_apr.shape)

C:\Users\luzhbin.aleksey\Desktop\analytics\Overdue_apr
(81007, 7)


**Загрузка данных: должники в марте**

In [147]:
%cd ..
%cd Overdue_mar
path = glob('*.csv')

deb_mar = pd.read_csv(path[0], sep = ';',
                dtype = {'LoanSum': 'float'},
                names = ['Dog', 'FIO', 'LoanDate', 'Sum', 'Status', 'DaysOverdue', 'OverdueDate'],
                usecols = [0, 2, 12, 13, 14, 15, 16], encoding = 'cp1251')

for i in range(1, len(path)):
    tmp = pd.read_csv(path[i], sep = ';',
                dtype = {'LoanSum': 'float'},
                names = ['Dog', 'FIO', 'LoanDate', 'Sum', 'Status', 'DaysOverdue', 'OverdueDate'],
                usecols = [0, 2, 12, 13, 14, 15, 16], encoding = 'cp1251')
    deb_mar = deb_mar.append(tmp, ignore_index = True)

print(deb_mar.shape)

C:\Users\luzhbin.aleksey\Desktop\analytics
C:\Users\luzhbin.aleksey\Desktop\analytics\Overdue_mar
(66701, 7)


**Создание объединенной таблицы по должникам за два месяца**

In [148]:
deb_mar_apr = deb_apr.append(deb_mar, ignore_index = True)
deb_mar_apr_column = pd.DataFrame(list(set(deb_mar_apr.Dog)), columns = ['Dog'])
deb_mar_apr_column['Debitor'] = True
deb_mar_apr_column.shape

(17329, 2)

**Загрузка данных: все заявки и операции в марте**

In [149]:
%cd ..
zvk_mar = pd.read_csv('zvk_mar.csv', sep = ';',
                    skiprows = 1, decimal = ',',
                    dtype = {'LoanSum': 'float'},
                    names = ['Dog', 'DateDog', 'City', 'ProductType', 'Product', 'Status', 'LoanSum', 'Duration',
                             'Name', 'Gender', 'BD', 'NumOfLoansBefore', 'NumOfLoansPayed'],
                    usecols = [0, 1, 5, 6, 7, 8, 9, 10, 14, 16, 17, 27, 37], encoding = 'cp1251')

zvk_mar['Month'] = 'March'
print(zvk_mar.shape)

oper_mar = pd.read_csv('oper_mar.csv', sep = ';',
                    skiprows = 1, decimal = ',',
                    dtype = {'LoanSum': 'float'},
                    names = ['Type', 'FIO', 'Dog', 'Sum', 'Org'],
                    usecols = [0, 1, 2, 5, 12], encoding = 'cp1251')

print(oper_mar.shape)

C:\Users\luzhbin.aleksey\Desktop\analytics
(143575, 14)
(56949, 5)


**Загрузка данных: все заявки и операции в апреле**

In [150]:
zvk_apr = pd.read_csv('zvk_apr.csv', sep = ';',
                    skiprows = 1, decimal = ',',
                    dtype = {'LoanSum': 'float'},
                    names = ['Dog', 'DateDog', 'City', 'ProductType', 'Product', 'Status', 'LoanSum', 'Duration',
                             'Name', 'Gender', 'BD', 'NumOfLoansBefore', 'NumOfLoansPayed'],
                    usecols = [0, 1, 5, 6, 7, 8, 9, 10, 14, 16, 17, 27, 37], encoding = 'cp1251')

zvk_apr['Month'] = 'April'
print(zvk_apr.shape)

oper_apr = pd.read_csv('oper_apr.csv', sep = ';',
                    skiprows = 1, decimal = ',',
                    dtype = {'LoanSum': 'float'},
                    names = ['Type', 'FIO', 'Dog', 'Sum', 'Org'],
                    usecols = [0, 1, 2, 5, 12], encoding = 'cp1251')

print(oper_apr.shape)

(70146, 14)
(63290, 5)


**Создание объединенной таблицы по заявкам за два месяца (за исключением auto-lombard)**

In [151]:
zvk_mar_apr = zvk_apr.append(zvk_mar, ignore_index = True)
zvk_mar_apr = zvk_mar_apr[(zvk_mar_apr.Product != 'auto-lombard') & (pd.isnull(zvk_mar_apr.Dog) != True)]
zvk_mar_apr = zvk_mar_apr[(zvk_mar_apr.ProductType != 'ПСК ДОВЕРИЕ')\
                            & (zvk_mar_apr.ProductType != 'Для сотрудников (ФМ) от 2000 от 29.01.2019')\
                            & (zvk_mar_apr.ProductType != 'Для сотрудников (ФМ) от 10500 от 29.01.2019')\
                            & (zvk_mar_apr.ProductType != 'Для сотрудников (ФМ) от 15500 от 29.01.2019')]
zvk_mar_apr['ON'] = list(map(lambda x: 'ON' in x, zvk_mar_apr.Dog))
zvk_mar_apr.shape

(212475, 15)

**Создание объединенной таблицы по операциям за два месяца (только тип "Выдача займа")**

In [152]:
oper_mar_apr = oper_apr.append(oper_mar, ignore_index = True)
oper_mar_apr = oper_mar_apr[(oper_mar_apr.Type == 'Выдача займа') & (pd.isnull(oper_mar_apr.Dog) != True)]
oper_mar_apr.shape

(31035, 5)

**Left join-cлияние двух объединенных таблиц: заявки и операции (выданные займы)**

In [153]:
merged_mar_apr = pd.merge(left = zvk_mar_apr, right = oper_mar_apr, left_on = 'Dog', right_on = 'Dog', how = 'left')
merged_mar_apr.shape

(212475, 19)

In [154]:
merged_mar_apr.dropna(inplace = True)
merged_mar_apr['Duration'] = list((map(int, merged_mar_apr.Duration.values)))
merged_mar_apr.shape

(30735, 19)

**Left join-cлияние двух итоговых таблиц: выданные займы и должники**

In [155]:
merged_mar_apr_deb = pd.merge(left = merged_mar_apr, right = deb_mar_apr_column, left_on = 'Dog',
                                                                      right_on = 'Dog', how = 'left')
merged_mar_apr_deb.dropna(inplace = True)
merged_mar_apr_deb.shape

(8697, 20)

**Разделение первичных и повторных заемщиков (только выданные займы): в срезе online/offline и март/апрель**

In [156]:
perv_mar_all = merged_mar_apr[(merged_mar_apr.Month == 'March') & (merged_mar_apr.NumOfLoansBefore == 0)]
povtor_mar_all = merged_mar_apr[(merged_mar_apr.Month == 'March') & (merged_mar_apr.NumOfLoansBefore != 0)]
perv_apr_all = merged_mar_apr[(merged_mar_apr.Month == 'April') & (merged_mar_apr.NumOfLoansBefore == 0)]
povtor_apr_all = merged_mar_apr[(merged_mar_apr.Month == 'April') & (merged_mar_apr.NumOfLoansBefore != 0)]
perv_mar_all_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'March') & (merged_mar_apr_deb.NumOfLoansBefore == 0)]
povtor_mar_all_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'March') & (merged_mar_apr_deb.NumOfLoansBefore != 0)]
perv_apr_all_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'April') & (merged_mar_apr_deb.NumOfLoansBefore == 0)]
povtor_apr_all_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'April') & (merged_mar_apr_deb.NumOfLoansBefore != 0)]

perv_mar_on = merged_mar_apr[(merged_mar_apr.Month == 'March') & (merged_mar_apr.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr.ON == True)]
povtor_mar_on = merged_mar_apr[(merged_mar_apr.Month == 'March') & (merged_mar_apr.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr.ON == True)]
perv_apr_on = merged_mar_apr[(merged_mar_apr.Month == 'April') & (merged_mar_apr.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr.ON == True)]
povtor_apr_on = merged_mar_apr[(merged_mar_apr.Month == 'April') & (merged_mar_apr.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr.ON == True)]
perv_mar_on_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'March') & (merged_mar_apr_deb.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr_deb.ON == True)]
povtor_mar_on_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'March') & (merged_mar_apr_deb.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr_deb.ON == True)]
perv_apr_on_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'April') & (merged_mar_apr_deb.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr_deb.ON == True)]
povtor_apr_on_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'April') & (merged_mar_apr_deb.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr_deb.ON == True)]

perv_mar_off = merged_mar_apr[(merged_mar_apr.Month == 'March') & (merged_mar_apr.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr.ON == False)]
povtor_mar_off = merged_mar_apr[(merged_mar_apr.Month == 'March') & (merged_mar_apr.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr.ON == False)]
perv_apr_off = merged_mar_apr[(merged_mar_apr.Month == 'April') & (merged_mar_apr.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr.ON == False)]
povtor_apr_off = merged_mar_apr[(merged_mar_apr.Month == 'April') & (merged_mar_apr.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr.ON == False)]
perv_mar_off_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'March') & (merged_mar_apr_deb.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr_deb.ON == False)]
povtor_mar_off_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'March') & (merged_mar_apr_deb.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr_deb.ON == False)]
perv_apr_off_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'April') & (merged_mar_apr_deb.NumOfLoansBefore == 0)\
                                                                                & (merged_mar_apr_deb.ON == False)]
povtor_apr_off_deb = merged_mar_apr_deb[(merged_mar_apr_deb.Month == 'April') & (merged_mar_apr_deb.NumOfLoansBefore != 0)\
                                                                                & (merged_mar_apr_deb.ON == False)]

**Вывод аналитики**

In [157]:
print('Первичные заемщики в марте:')

print('Кол-во займов: {} ({:.2f}%)'.format(len(perv_mar_all.Dog),
                len(perv_mar_all.Dog) * 100 / (len(perv_mar_all.Dog) + len(povtor_mar_all.Dog))))
print('в т.ч. online: {} ({:.2f}%)'\
                .format(len(perv_mar_on.Dog), len(perv_mar_on.Dog) * 100 / (len(perv_mar_all.Dog))))
print('в т.ч. offline: {} ({:.2f}%)'\
                .format(len(perv_mar_off.Dog), len(perv_mar_off.Dog) * 100 / (len(perv_mar_all.Dog))))

print('Cредняя сумма займа: {:.0f}'.format(np.mean(perv_mar_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.mean(perv_mar_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.mean(perv_mar_off.Sum)))

print('Медианная сумма займа: {:.0f}'.format(np.median(perv_mar_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.median(perv_mar_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.median(perv_mar_off.Sum)))

print('Cредний (первоначальный) срок займа: {:.0f}'.format(np.mean(perv_mar_all.Duration)))
print('в т.ч. online: {:.0f}'.format(np.mean(perv_mar_on.Duration)))
print('в т.ч. offline: {:.0f}'.format(np.mean(perv_mar_off.Duration)))

print('Конверсия в выдачу (шт.): {:.2f}%'.format(len(perv_mar_all.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'March')\
                                                & (zvk_mar_apr.NumOfLoansBefore == 0)])))
print('в т.ч. online: {:.2f}%'.format(len(perv_mar_on.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'March')\
                                                & (zvk_mar_apr.NumOfLoansBefore == 0)\
                                                & (zvk_mar_apr.ON == True)])))
print('в т.ч. offline: {:.2f}%'.format(len(perv_mar_off.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'March')\
                                                & (zvk_mar_apr.NumOfLoansBefore == 0)\
                                                & (zvk_mar_apr.ON == False)])))

print('Выход на просрочку (шт.): {:.2f}%'.format(len(perv_mar_all_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'March')\
                                                & (merged_mar_apr.NumOfLoansBefore == 0)])))
print('в т.ч. online: {:.2f}%'.format(len(perv_mar_on_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'March')\
                                                & (merged_mar_apr.NumOfLoansBefore == 0)\
                                                & (merged_mar_apr.ON == True)])))
print('в т.ч. offline: {:.2f}%'.format(len(perv_mar_off_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'March')\
                                                & (merged_mar_apr.NumOfLoansBefore == 0)\
                                                & (merged_mar_apr.ON == False)])))

Первичные заемщики в марте:
Кол-во займов: 5185 (35.22%)
в т.ч. online: 3308 (63.80%)
в т.ч. offline: 1877 (36.20%)
Cредняя сумма займа: 6575
в т.ч. online: 5741
в т.ч. offline: 8043
Медианная сумма займа: 5000
в т.ч. online: 5000
в т.ч. offline: 7000
Cредний (первоначальный) срок займа: 22
в т.ч. online: 23
в т.ч. offline: 19
Конверсия в выдачу (шт.): 4.04%
в т.ч. online: 7.42%
в т.ч. offline: 2.24%
Выход на просрочку (шт.): 52.56%
в т.ч. online: 52.72%
в т.ч. offline: 52.26%


In [158]:
print('Повторные заемщики в марте:')

print('Кол-во займов: {} ({:.2f}%)'.format(len(povtor_mar_all.Dog),
                len(povtor_mar_all.Dog) * 100 / (len(perv_mar_all.Dog) + len(povtor_mar_all.Dog))))
print('в т.ч. online: {} ({:.2f}%)'\
                .format(len(povtor_mar_on.Dog), len(povtor_mar_on.Dog) * 100 / (len(povtor_mar_all.Dog))))
print('в т.ч. offline: {} ({:.2f}%)'\
                .format(len(povtor_mar_off.Dog), len(povtor_mar_off.Dog) * 100 / (len(povtor_mar_all.Dog))))

print('Cредняя сумма займа: {:.0f}'.format(np.mean(povtor_mar_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.mean(povtor_mar_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.mean(povtor_mar_off.Sum)))

print('Медианная сумма займа: {:.0f}'.format(np.median(povtor_mar_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.median(povtor_mar_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.median(povtor_mar_off.Sum)))

print('Cредний (первоначальный) срок займа: {:.0f}'.format(np.mean(povtor_mar_all.Duration)))
print('в т.ч. online: {:.0f}'.format(np.mean(povtor_mar_on.Duration)))
print('в т.ч. offline: {:.0f}'.format(np.mean(povtor_mar_off.Duration)))

print('Конверсия в выдачу (шт.): {:.2f}%'.format(len(povtor_mar_all.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'March')\
                                                & (zvk_mar_apr.NumOfLoansBefore != 0)])))
print('в т.ч. online: {:.2f}%'.format(len(povtor_mar_on.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'March')\
                                                & (zvk_mar_apr.NumOfLoansBefore != 0)\
                                                & (zvk_mar_apr.ON == True)])))
print('в т.ч. offline: {:.2f}%'.format(len(povtor_mar_off.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'March')\
                                                & (zvk_mar_apr.NumOfLoansBefore != 0)\
                                                & (zvk_mar_apr.ON == False)])))

print('Выход на просрочку (шт.): {:.2f}%'.format(len(povtor_mar_all_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'March')\
                                                & (merged_mar_apr.NumOfLoansBefore != 0)])))
print('в т.ч. online: {:.2f}%'.format(len(povtor_mar_on_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'March')\
                                                & (merged_mar_apr.NumOfLoansBefore != 0)\
                                                & (merged_mar_apr.ON == True)])))
print('в т.ч. offline: {:.2f}%'.format(len(povtor_mar_off_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'March')\
                                                & (merged_mar_apr.NumOfLoansBefore != 0)\
                                                & (merged_mar_apr.ON == False)])))

Повторные заемщики в марте:
Кол-во займов: 9536 (64.78%)
в т.ч. online: 6309 (66.16%)
в т.ч. offline: 3227 (33.84%)
Cредняя сумма займа: 11813
в т.ч. online: 13092
в т.ч. offline: 9312
Медианная сумма займа: 8000
в т.ч. online: 10000
в т.ч. offline: 6000
Cредний (первоначальный) срок займа: 23
в т.ч. online: 23
в т.ч. offline: 23
Конверсия в выдачу (шт.): 66.44%
в т.ч. online: 63.52%
в т.ч. offline: 73.01%
Выход на просрочку (шт.): 42.52%
в т.ч. online: 50.71%
в т.ч. offline: 26.53%


In [159]:
print('Первичные заемщики в апреле:')

print('Кол-во займов: {} ({:.2f}%)'.format(len(perv_apr_all.Dog),
                len(perv_apr_all.Dog) * 100 / (len(perv_apr_all.Dog) + len(povtor_apr_all.Dog))))
print('в т.ч. online: {} ({:.2f}%)'\
                .format(len(perv_apr_on.Dog), len(perv_apr_on.Dog) * 100 / (len(perv_apr_all.Dog))))
print('в т.ч. offline: {} ({:.2f}%)'\
                .format(len(perv_apr_off.Dog), len(perv_apr_off.Dog) * 100 / (len(perv_apr_all.Dog))))

print('Cредняя сумма займа: {:.0f}'.format(np.mean(perv_apr_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.mean(perv_apr_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.mean(perv_apr_off.Sum)))

print('Медианная сумма займа: {:.0f}'.format(np.median(perv_apr_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.median(perv_apr_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.median(perv_apr_off.Sum)))

print('Cредний (первоначальный) срок займа: {:.0f}'.format(np.mean(perv_apr_all.Duration)))
print('в т.ч. online: {:.0f}'.format(np.mean(perv_apr_on.Duration)))
print('в т.ч. offline: {:.0f}'.format(np.mean(perv_apr_off.Duration)))

print('Конверсия в выдачу (шт.): {:.2f}%'.format(len(perv_apr_all.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'April')\
                                                & (zvk_mar_apr.NumOfLoansBefore == 0)])))
print('в т.ч. online: {:.2f}%'.format(len(perv_apr_on.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'April')\
                                                & (zvk_mar_apr.NumOfLoansBefore == 0)\
                                                & (zvk_mar_apr.ON == True)])))
print('в т.ч. offline: {:.2f}%'.format(len(perv_apr_off.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'April')\
                                                & (zvk_mar_apr.NumOfLoansBefore == 0)\
                                                & (zvk_mar_apr.ON == False)])))

print('Выход на просрочку (шт.): {:.2f}% (непоказательно)'.format(len(perv_apr_all_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'April')\
                                                & (merged_mar_apr.NumOfLoansBefore == 0)])))

Первичные заемщики в апреле:
Кол-во займов: 5835 (36.44%)
в т.ч. online: 4461 (76.45%)
в т.ч. offline: 1374 (23.55%)
Cредняя сумма займа: 6359
в т.ч. online: 6048
в т.ч. offline: 7368
Медианная сумма займа: 5000
в т.ч. online: 5000
в т.ч. offline: 6000
Cредний (первоначальный) срок займа: 19
в т.ч. online: 19
в т.ч. offline: 18
Конверсия в выдачу (шт.): 10.52%
в т.ч. online: 10.07%
в т.ч. offline: 12.33%
Выход на просрочку (шт.): 19.47% (непоказательно)


In [160]:
print('Повторные заемщики в апреле:')

print('Кол-во займов: {} ({:.2f}%)'.format(len(povtor_apr_all.Dog),
                len(povtor_apr_all.Dog) * 100 / (len(perv_apr_all.Dog) + len(povtor_apr_all.Dog))))
print('в т.ч. online: {} ({:.2f}%)'\
                .format(len(povtor_apr_on.Dog), len(povtor_apr_on.Dog) * 100 / (len(povtor_apr_all.Dog))))
print('в т.ч. offline: {} ({:.2f}%)'\
                .format(len(povtor_apr_off.Dog), len(povtor_apr_off.Dog) * 100 / (len(povtor_apr_all.Dog))))

print('Cредняя сумма займа: {:.0f}'.format(np.mean(povtor_apr_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.mean(povtor_apr_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.mean(povtor_apr_off.Sum)))

print('Медианная сумма займа: {:.0f}'.format(np.median(povtor_apr_all.Sum)))
print('в т.ч. online: {:.0f}'.format(np.median(povtor_apr_on.Sum)))
print('в т.ч. offline: {:.0f}'.format(np.median(povtor_apr_off.Sum)))

print('Cредний (первоначальный) срок займа: {:.0f}'.format(np.mean(povtor_apr_all.Duration)))
print('в т.ч. online: {:.0f}'.format(np.mean(povtor_apr_on.Duration)))
print('в т.ч. offline: {:.0f}'.format(np.mean(povtor_apr_off.Duration)))

print('Конверсия в выдачу (шт.): {:.2f}%'.format(len(povtor_apr_all.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'April')\
                                                & (zvk_mar_apr.NumOfLoansBefore != 0)])))
print('в т.ч. online: {:.2f}%'.format(len(povtor_apr_on.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'April')\
                                                & (zvk_mar_apr.NumOfLoansBefore != 0)\
                                                & (zvk_mar_apr.ON == True)])))
print('в т.ч. offline: {:.2f}%'.format(len(povtor_apr_off.Dog) *\
                                                100 / len(zvk_mar_apr.Dog[(zvk_mar_apr.Month == 'April')\
                                                & (zvk_mar_apr.NumOfLoansBefore != 0)\
                                                & (zvk_mar_apr.ON == False)])))

print('Выход на просрочку (шт.): {:.2f}% (непоказательно)'.format(len(povtor_apr_all_deb.Dog) *\
                                                100 / len(merged_mar_apr.Dog[(merged_mar_apr.Month == 'April')\
                                                & (merged_mar_apr.NumOfLoansBefore != 0)])))

Повторные заемщики в апреле:
Кол-во займов: 10179 (63.56%)
в т.ч. online: 6787 (66.68%)
в т.ч. offline: 3392 (33.32%)
Cредняя сумма займа: 10525
в т.ч. online: 11174
в т.ч. offline: 9224
Медианная сумма займа: 8000
в т.ч. online: 8000
в т.ч. offline: 6000
Cредний (первоначальный) срок займа: 23
в т.ч. online: 23
в т.ч. offline: 23
Конверсия в выдачу (шт.): 71.79%
в т.ч. online: 67.28%
в т.ч. offline: 82.91%
Выход на просрочку (шт.): 7.67% (непоказательно)
